In [10]:
import pandas as pd
from gensim.models import FastText
import re
import numpy as np
from libsvm import svmutil
import libsvm
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.stats import kendalltau

In [2]:
# data["Category"].replace({"Tech": "Technology","-":"Uncategorized", "Sports":"Sport"}, inplace=True)
model_summary = FastText.load("models-4.1(a)/model_summary.bin")


In [3]:
data = pd.read_excel("NewsHeadlines_01012019_30092019.xlsx")
data.columns =[column.replace(" ", "_") for column in data.columns]
data.query(' Post_Type == "photo" or Post_Type == "video" or Post_Type == "link" ', inplace=True)
data.reset_index(inplace=True, drop=True)


In [4]:
post_types = set(data["Post_Type"].tolist())
headlines= []
for sent_str in data["Headline"]:
#     sent_str = re.sub(r"'", "", str(sent_str).lower())
    tokens = re.sub(r"[^a-z0-9]+", " ", str(sent_str).lower()).split()
    if(tokens == []):
        tokens = "nan"
    headlines.append(tokens)
    
cat_vec = {}
for c in post_types:
    cat_vec[c] = model_summary[c]    
    
head_vec = []
n = len(headlines)
for h in range(n):
    sum1 = np.array([0.0 for i in range(100)])
    for tok in headlines[h]:
        sum1+=model_summary[tok]
#     print(sum1)
    sum1 = sum1/len(headlines[h])
#     print(sum1)
    sum1 = np.concatenate((sum1,cat_vec[data["Post_Type"][h]]))
    head_vec.append(sum1)

/home/prachi/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
/home/prachi/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [6]:
feat_len = int(0.8*len(head_vec))
train = head_vec[:feat_len]
res_train = list(np.log2(np.array(data["Like_Count"][:feat_len].tolist())+1))
test = head_vec[feat_len::]
res_test = list(np.log2(np.array(data["Like_Count"][feat_len:].tolist())+1))
print(len(train), len(res_train))

38064 38064


In [7]:
# model_svm = svmutil.svm_train(res_train, train,'-s 4 -t 2 -c 2')

In [19]:
svmutil.svm_save_model("model_c_reg.bin",model_svm)

In [8]:
y_reg = svmutil.svm_predict(res_test, test, model_svm)

Mean squared error = 5.26695 (regression)
Squared correlation coefficient = 0.095472 (regression)


In [11]:
r2 = r2_score(res_test,y_reg[0])
r2

-0.1669129819160884

In [13]:
mae = mean_absolute_error(res_test, y_reg[0])
mae

1.788310632601459

In [14]:
kt = kendalltau(res_test, y_reg[0])
kt

KendalltauResult(correlation=0.18798736195967408, pvalue=2.3026195471272275e-165)

In [15]:
y_full = svmutil.svm_predict(res_test+res_train, test+train, model_svm)

Mean squared error = 3.3811 (regression)
Squared correlation coefficient = 0.133888 (regression)


In [16]:
r2 = r2_score(res_test+res_train,y_full[0])
r2

0.12518770453790495

In [17]:
mae = mean_absolute_error(res_test+res_train, y_full[0])
mae

1.4494976067423697

In [18]:
kt = kendalltau(res_test+res_train, y_full[0])
kt

KendalltauResult(correlation=0.24028299649081708, pvalue=0.0)

In [20]:
y_tr = svmutil.svm_predict(res_train, train, model_svm)

Mean squared error = 2.90959 (regression)
Squared correlation coefficient = 0.167139 (regression)


In [24]:
r2 = r2_score(res_train,y_tr[0])
r2

0.16429139248593794

In [22]:
mae = mean_absolute_error(res_train, y_tr[0])
mae

1.364785449136733

In [23]:
kt = kendalltau(res_train, y_tr[0])
kt

KendalltauResult(correlation=0.26845595610803163, pvalue=0.0)

In [1]:
model_svm_reg = svmutil.svm_train(res_train+res_test, train+test,'-s 4 -v 10 -t 2 -c 2')

NameError: name 'svmutil' is not defined